In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
from keras.utils import multi_gpu_model, multi_gpu_utils
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")
SIZE = 512

Using TensorFlow backend.


In [2]:
# Load dataset info
path_to_train = '../../Human_Protein_Atlas/input/train/'
data = pd.read_csv('../../Human_Protein_Atlas/input/train.csv')

rare_classes = [8,9,10,15,20,24,27]

train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

class data_generator:

    @staticmethod
    def create_train(dataset_info, batch_size, shape, augument=True, heavy_augment_rares=True, oversample_factor = 0):
        assert shape[2] == 3



        if oversample_factor > 0:

            rare_dataset_info = np.array([item for item in dataset_info if np.isin(item['labels'], rare_classes).any()])
            #rare_dataset_info = shuffle(rare_dataset_info)
            for i in range(oversample_factor):
            #dataset_info
                dataset_info = np.append(dataset_info,rare_dataset_info)
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(X_train_batch[i]['path'], shape)
                    #image = preprocess_input(image)
                    rare = np.isin(X_train_batch[i]['labels'], rare_classes).any()

                    if augument:
                        if heavy_augment_rares and rare:
                            image = data_generator.heavy_augment(image)
                        else:
                            image = data_generator.augment(image)

                    batch_images.append(image)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    @staticmethod
    def load_image(path, shape):
        image = cv2.imread(path + '.png', cv2.IMREAD_UNCHANGED)
        return image

    @staticmethod
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug

    @staticmethod
    def heavy_augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(scale=(0.5,2.0)),
                iaa.Affine(shear=15),
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=35),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Affine(translate_percent=0.1),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.Noop()
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [19]:
train_dataset_info

array([{'path': '../../Human_Protein_Atlas/input/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../../Human_Protein_Atlas/input/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../../Human_Protein_Atlas/input/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../../Human_Protein_Atlas/input/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../../Human_Protein_Atlas/input/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../../Human_Protein_Atlas/input/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [15]:
rare_classes = ["Endosomes", "Lysosomes", "Rods & rings"]

In [ ]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionResNetV2(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [4]:
epochs = 30; batch_size = 16
checkpoint = ModelCheckpoint('../../Human_Protein_Atlas/working/InceptionV3_512.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, 
                                   verbose=1, mode='auto', epsilon=0.00001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [5]:
# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.1, random_state=18)

# create train and valid datagens
train_generator = data_generator.create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = data_generator.create_train(
    train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

In [6]:
with tf.device('/cpu:0'):
    input_shape=(SIZE,SIZE,3)
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(28, activation='sigmoid')(x)
    model = Model(input_tensor, output)



In [7]:
model = multi_gpu_model(model, gpus=2)


In [3]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
        np.array(image_red_ch), 
        np.array(image_green_ch), 
        np.array(image_blue_ch)), -1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug

In [8]:
for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True


model.compile(
    loss='binary_crossentropy', 
    optimizer=Adam(1e-03),
    metrics=['acc'])
# model.summary()
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

Epoch 1/2
1748/1748 [==============================] - 1433s 820ms/step - loss: 0.1675 - acc: 0.9453 - val_loss: 0.1668 - val_acc: 0.9448
Epoch 2/2
1748/1748 [==============================] - 745s 426ms/step - loss: 0.1531 - acc: 0.9487 - val_loss: 0.1541 - val_acc: 0.9497


In [9]:
epochs = 30

# train all layers
for layer in model.layers:
    layer.trainable = True
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=1e-5),
            metrics=['accuracy'])
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/30
1748/1748 [==============================] - 806s 461ms/step - loss: 0.1419 - acc: 0.9510 - val_loss: 0.1325 - val_acc: 0.9536

Epoch 00001: val_loss improved from inf to 0.13249, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 2/30
 647/1748 [==========>...................] - ETA: 7:13 - loss: 0.1400 - acc: 0.9515

KeyboardInterrupt: 

In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/30
1748/1748 [==============================] - 766s 438ms/step - loss: 0.1395 - acc: 0.9518 - val_loss: 0.1309 - val_acc: 0.9541

Epoch 00001: val_loss improved from 0.13249 to 0.13087, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 2/30
1748/1748 [==============================] - 764s 437ms/step - loss: 0.1380 - acc: 0.9524 - val_loss: 0.1300 - val_acc: 0.9542

Epoch 00002: val_loss improved from 0.13087 to 0.12996, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 3/30
1748/1748 [==============================] - 765s 437ms/step - loss: 0.1369 - acc: 0.9524 - val_loss: 0.1294 - val_acc: 0.9543

Epoch 00003: val_loss improved from 0.12996 to 0.12938, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 4/30
1748/1748 [==============================] - 767s 439ms/step - loss: 0.1355 - acc: 0.9529 - val_loss: 0.1278 - val_acc: 0.9552

Epoch 00004: val_loss improved from 0.12938 to 0.12784, saving model 

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/30
1748/1748 [==============================] - 772s 442ms/step - loss: 0.1084 - acc: 0.9620 - val_loss: 0.1044 - val_acc: 0.9631

Epoch 00001: val_loss improved from 0.10448 to 0.10440, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 2/30
1748/1748 [==============================] - 766s 438ms/step - loss: 0.1090 - acc: 0.9618 - val_loss: 0.1033 - val_acc: 0.9637

Epoch 00002: val_loss improved from 0.10440 to 0.10335, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 3/30
1748/1748 [==============================] - 768s 439ms/step - loss: 0.1070 - acc: 0.9626 - val_loss: 0.1037 - val_acc: 0.9638

Epoch 00003: val_loss did not improve from 0.10335
Epoch 4/30
1748/1748 [==============================] - 766s 438ms/step - loss: 0.1071 - acc: 0.9622 - val_loss: 0.1014 - val_acc: 0.9640

Epoch 00004: val_loss improved from 0.10335 to 0.10143, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 5/30
1748/17

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/30
1748/1748 [==============================] - 762s 436ms/step - loss: 0.0894 - acc: 0.9682 - val_loss: 0.0912 - val_acc: 0.9677

Epoch 00001: val_loss improved from 0.09227 to 0.09122, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 2/30
1748/1748 [==============================] - 764s 437ms/step - loss: 0.0887 - acc: 0.9685 - val_loss: 0.0911 - val_acc: 0.9682

Epoch 00002: val_loss improved from 0.09122 to 0.09109, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 3/30
1748/1748 [==============================] - 767s 439ms/step - loss: 0.0882 - acc: 0.9687 - val_loss: 0.0910 - val_acc: 0.9683

Epoch 00003: val_loss improved from 0.09109 to 0.09103, saving model to ../../Human_Protein_Atlas/working/InceptionV3_512.h5
Epoch 4/30
1748/1748 [==============================] - 769s 440ms/step - loss: 0.0877 - acc: 0.9690 - val_loss: 0.0922 - val_acc: 0.9675

Epoch 00004: val_loss did not improve from 0.09103
Epoch 5/30
1748/17

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1748/1748 [==============================] - 764s 437ms/step - loss: 0.0769 - acc: 0.9723 - val_loss: 0.0877 - val_acc: 0.9696

Epoch 00026: val_loss did not improve from 0.08756
Epoch 27/30
1268/1748 [====================>.........] - ETA: 3:11 - loss: 0.0765 - acc: 0.9724

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1748/1748 [==============================] - 762s 436ms/step - loss: 0.0766 - acc: 0.9726 - val_loss: 0.0877 - val_acc: 0.9694

Epoch 00030: val_loss did not improve from 0.08735


In [13]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/30
1748/1748 [==============================] - 763s 436ms/step - loss: 0.0762 - acc: 0.9727 - val_loss: 0.0876 - val_acc: 0.9695

Epoch 00001: val_loss did not improve from 0.08735
Epoch 2/30
1748/1748 [==============================] - 762s 436ms/step - loss: 0.0767 - acc: 0.9726 - val_loss: 0.0877 - val_acc: 0.9692

Epoch 00002: val_loss did not improve from 0.08735
Epoch 3/30
1748/1748 [==============================] - 768s 439ms/step - loss: 0.0760 - acc: 0.9727 - val_loss: 0.0877 - val_acc: 0.9694

Epoch 00003: val_loss did not improve from 0.08735
Epoch 4/30
1748/1748 [==============================] - 767s 439ms/step - loss: 0.0764 - acc: 0.9725 - val_loss: 0.0879 - val_acc: 0.9694

Epoch 00004: val_loss did not improve from 0.08735
Epoch 5/30
1748/1748 [==============================] - 771s 441ms/step - loss: 0.0757 - acc: 0.9729 - val_loss: 0.0875 - val_acc: 0.9696

Epoch 00005: val_loss did not improve from 0.08735
Epoch 6/30
1748/1748 [============================